# ex191124

## 初期設定

In [1]:
from google.colab import drive
drive.mount('gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [2]:
pathGDrive = 'gdrive/My Drive/ex191121'
!ls -l 'gdrive/My Drive/ex191121'
#!cat 'gdrive/My Drive/ex191121/ex191124_cnnAE2.py'

total 749901
-rw------- 1 root root 755569425 Nov 21 13:02 data48x48.npz
-rw------- 1 root root     18880 Nov 22 01:04 ex1121-AE.ipynb
-rw------- 1 root root   7078479 Nov 22 00:00 ex1121-params.pickle
-rw------- 1 root root     23665 Nov 22 09:51 ex1122-cnnAE.ipynb
-rw------- 1 root root    939965 Nov 22 09:50 ex1122-params.pickle
-rw------- 1 root root      3339 Nov 24 22:35 ex191124_cnnAE2.py
-rw------- 1 root root      2974 Nov 24 14:35 ex191124_cnnAE3.py
-rw------- 1 root root      2777 Nov 24 10:47 ex191124_cnnAE.py
-rw------- 1 root root      1708 Nov 24 07:59 ex191124_data.py
-rw------- 1 root root     23996 Nov 25 00:17 ex191124.ipynb
-rw------- 1 root root    939943 Nov 24 10:52 ex191124-params.pickle
-rw------- 1 root root     64370 Nov 22 09:51 hoge1122_h0100.png
-rw------- 1 root root     68337 Nov 22 09:45 hoge1122_h0200.png
-rw------- 1 root root     64377 Nov 24 11:00 hoge191124_08_08_0100.png
-rw------- 1 root root     71406 Nov 25 00:00 hoge191124_L_08_16_0400.png
-rw

In [0]:
fnData = pathGDrive + '/data48x48.npz'

In [0]:
import numpy as np
import cv2
import datetime
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
### GoogleDrive上の hoge.py を import できるように
#
sys.path.append(pathGDrive)
import ex191124_data as data
import ex191124_cnnAE as network
#import ex191124_cnnAE2 as network
#import ex191124_cnnAE3 as network

In [6]:
### device
#
use_gpu_if_available = True
if use_gpu_if_available and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('# using', device)

# using cuda


In [7]:
### loading and setting the data
#
XL, XT, Xmean = data.loadData(path=fnData, forCNN=True)
XL = torch.from_numpy(data.HWC2CHW(XL)).to(device)
XT = torch.from_numpy(data.HWC2CHW(XT)).to(device)
print(XL.shape, XT.shape, Xmean.shape, XL.dtype)
NL = XL.shape[0]
NT = XT.shape[0]

batchsize = 128
bindexL = data.makeBatchIndex(NL, batchsize)
nbatchL = bindexL.shape[0]
bindexT = data.makeBatchIndex(NT, batchsize)
nbatchT = bindexT.shape[0]

(10378, 48, 48, 3) (4000, 48, 48, 3) (48, 48, 3)
torch.Size([10378, 3, 48, 48]) torch.Size([4000, 3, 48, 48]) (48, 48, 3) torch.float32


## 学習

In [0]:
# ストッパー
hoge

NameError: ignored

In [8]:
### initializing the network
#
torch.manual_seed(0)
net = network.AE(XL.shape[1:], 16, 32, 800)
#net = network.AE(XL.shape[1:], 16, 32, 64, 100)
model = net.to(device)
print(net)
#optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(optimizer)

Y = model(XL[bindexL[0], :])
print(Y.shape)

# input: torch.Size([1, 3, 48, 48])
# conv1 output: torch.Size([1, 16, 24, 24])
# conv2 output: torch.Size([1, 32, 12, 12])
# conv2_shape: [32 12 12]
# flatten output: torch.Size([1, 4608])
# fc1 output: torch.Size([1, 800])
# fc2 output: torch.Size([1, 4608])
# reshaped output: torch.Size([1, 32, 12, 12])
# tconv1 output: torch.Size([1, 16, 24, 24])
# tconv2 output: torch.Size([1, 3, 48, 48])
AE(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (fc1): Sequential(
    (0): Linear(in_features=4608, out_features=800, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (fc2): Sequential(
    (0): Linear(in_features=800, out_features=4608, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (tconv1): Sequential(
    (0): ConvTranspose2d(32, 16, ker

In [9]:
### training
#
nitr = 20000
nd = 0
start = datetime.datetime.now()

for i in range(nitr):

    if (i < 500 and i % 100 == 0) or (i % 500 == 0):
        model.eval()
        msqe = network.evaluate(model, XL, XL, bindexL)
        print(i, nd, msqe)

    model.train()
    ib = np.random.randint(0, nbatchL)
    ii = np.where(bindexL[ib, :])[0]
    optimizer.zero_grad()
    output = model(XL[ii, :])
    loss = F.mse_loss(output, XL[ii])
    loss.backward()
    optimizer.step()

    nd += ii.shape[0]

model.eval()
msqe = network.evaluate(model, XL, XL, bindexL)
print(nitr, nd, msqe)

print('# elapsed time: ', datetime.datetime.now() - start)
    

0 0 458.0891799265122
100 12682 178.04560087199937
200 25246 136.08247963314824
300 37456 126.28543684234002
400 50138 114.0836559684549
500 62938 102.67882252162214
1000 125876 73.46289101904081
1500 188932 63.57329178371051
2000 252224 58.667135162559354
2500 315398 51.27083509867535
3000 379044 47.628638799214876
3500 442690 44.708914223974396
4000 506336 42.16189705169966
4500 569274 40.67831299745593
5000 632566 51.60734439011046
5500 696330 38.17056876943621
6000 759150 37.34187561258642
6500 822560 35.55624323190809
7000 885144 35.55890752410448
7500 948318 33.148096831140315
8000 1011728 32.78889367704011
8500 1074548 33.38356977291111
9000 1137486 32.03205625159498
9500 1200306 32.16237010356663
10000 1263716 29.619125178012343
10500 1327008 29.401830375849293
11000 1390654 28.709502560712394
11500 1453828 32.191741849260076
12000 1517238 30.58930309513994
12500 1580530 28.091189161158407
13000 1643822 27.87715818942179
13500 1707350 29.436117191722705
14000 1771114 27.8378257

In [0]:
print(msqe)

28.71601815774138


In [0]:
### saving the model
#
fnModel = pathGDrive + '/ex191124-params.pickle'
with open(fnModel, mode = 'wb') as f:
    torch.save(model.state_dict(), f)
    print('# The model is saved to ', fnModel)

## テスト

In [0]:
!ls

In [0]:
fnModel = pathGDrive + '/ex191124-params.pickle'

### loading the network
#
torch.manual_seed(0)
net = network.AE(XL.shape[1:], 8, 8, 100)
with open(fnModel, mode = 'rb') as f:
    net.load_state_dict(torch.load(f))
    model = net.to(device)
print(net)

In [0]:
model.eval()
msqe = network.evaluate(model, XT, XT, bindexT)
print(msqe)

34.05712329101563


In [0]:
idx = np.array([[0, 100, 200, 300], [400, 500, 600, 700], [800, 900, 1000, 1100]])

In [0]:
#ZZ = XL
ZZ = model(XL)
ZZ = ZZ.cpu().detach().numpy()
ZZ = data.CHW2HWC(ZZ)
print(ZZ.shape)
img = data.montage(ZZ, Xmean, idx)
print(ZZ.shape, img.shape)

(10378, 48, 48, 3)
(10378, 48, 48, 3) (160, 212, 3)


In [0]:
fnResult = pathGDrive + '/hoge191124_L.png'
cv2.imwrite(fnResult, img)

True

In [0]:
#ZZ = XT
ZZ = model(XT)
ZZ = ZZ.cpu().detach().numpy()
ZZ = data.CHW2HWC(ZZ)
print(ZZ.shape)
img = data.montage(ZZ, Xmean, idx)
print(ZZ.shape, img.shape)

(4000, 48, 48, 3)
(4000, 48, 48, 3) (160, 212, 3)


In [0]:
fnResult = pathGDrive + '/hoge191124_T.png'
cv2.imwrite(fnResult, img)

True

## memo

### results with ex191124_cnnAE.py

net = network.AE(XL.shape[1:], 8, 16, 400)
- train: 51.42749410714917
- test: 54.27171084594727

net = network.AE(XL.shape[1:], 8, 16, 800)
- train: 34.40051437145622
- test: 37.135433563232425

net = network.AE(XL.shape[1:], 8, 16, 1200)
- train: 28.422655784502542
- test: 32.46093212890625

net = network.AE(XL.shape[1:], 8, 16, 1600)
- train: 28.71601815774138
- test: 34.05712329101563

net = network.AE(XL.shape[1:], 16, 32, 100)
- train: 94.58950835070092
- test: 112.25799243164063

net = network.AE(XL.shape[1:], 16, 32, 200)
- train: 68.24375702921745
- test:  81.6006773071289

net = network.AE(XL.shape[1:], 16, 32, 400)
- train: 44.07497419091543
- test:  52.83317532348633

net = network.AE(XL.shape[1:], 16, 32, 800)
- train: 25.287875205801043
- test: 35.361615844726565

net = network.AE(XL.shape[1:], 16, 32, 1200)
- train: 20.619673221137486
- test: 37.471118591308596

net = network.AE(XL.shape[1:], 16, 32, 1600)
- train: 19.10102583700854
- test: 48.35784045410156

net = network.AE(XL.shape[1:], 32, 64, 100)
- train: 81.34184966470539
- test: 126.97538305664062

net = network.AE(XL.shape[1:], 32, 64, 200)
- train: 55.396759436065736
- test: 97.164984375

net = network.AE(XL.shape[1:], 32, 64, 400)
- train: 34.1949384023652
- test:  70.30912493896484

net = network.AE(XL.shape[1:], 32, 64, 800)
- train: 20.19049364561565
- test: 65.24725354003907

### results with ex191124_cnnAE2.py

net = network.AE(XL.shape[1:], 16, 32, 64, 100)
- train: 95.94117693001421
- test: 105.62162536621094

net = network.AE(XL.shape[1:], 16, 32, 64, 200)
- train: 69.90124503485062
- test: 74.47183520507812

net = network.AE(XL.shape[1:], 16, 32, 64, 400)
- train: 48.156987617920535
- test: 51.570237365722654

net = network.AE(XL.shape[1:], 16, 32, 64, 800)
- train: 38.49635803669628
- test: 41.484577331542965

net = network.AE(XL.shape[1:], 16, 32, 64, 1200)
- train:  34.68962041002451
- test: 38.88744055175781

net = network.AE(XL.shape[1:], 16, 32, 64, 1600)
- train: 32.85508997974701 
- test: 45.49587686157226


### results with ex191124_cnnAE3.py

conv1 と conv2 のみ BatchNorm2d

net = network.AE(XL.shape[1:], 16, 32, 400)
- train: 48.14324750622749
- test: 62.78153344726562

conv1, conv2, fc1, fc2 まで BatchNorm

net = network.AE(XL.shape[1:], 16, 32, 400)
- train: 41.40474721070865
- test: 80.49330340576172